<a href="https://colab.research.google.com/github/rimbourouphael/Devoir-NFP103/blob/main/Merge_sort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Devoir
L'objectif de cet exercice est de comparer le temps d'exécution de l'algorithme de tri par fusion (merge sort) lorsqu'il est exécuté de manière séquentielle, parallélisée avec des threads et parallélisée avec des processus.


Divisez votre liste en deux parties égales.
Utilisez deux threads pour trier chaque moitié de la liste simultanément.
Fusionnez ensuite les deux listes triées dans le thread principal.


Tout comme avec les threads, divisez votre liste en deux parties égales.
Utilisez deux processus pour trier chaque moitié de la liste simultanément.
Fusionnez ensuite les deux listes triées dans le processus principal.
:
Utilisez le module time de Python pour mesurer combien de temps chaque implémentation prend pour trier une liste de N éléments (par exemple, pour N = 10^4, 10^5, 10^6...).


Analysez et expliquez les résultats obtenus. Pourquoi l'une des méthodes est-elle plus rapide ou plus lente qu'une autre ? Comment le coût de la création de threads ou de processus affecte-t-il les résultats ? Quel impact le GIL (Global Interpreter Lock) de Python a-t-il sur les threads ?


Essayez d'utiliser plus de deux threads ou processus (par exemple, 4, 8, 16...) et observez comment cela affecte les performances.

# Merge sort séquentiel

In [1]:
def merge_sort(lst):
    # Si la liste est vide ou contient un seul élément, elle est déjà triée
    if len(lst) <= 1:
        return lst

    # Sinon, on divise la liste en deux moitiés
    middle = len(lst) // 2
    left = lst[:middle]
    right = lst[middle:]

    # On trie chaque moitié de manière récursive
    left = merge_sort(left)
    right = merge_sort(right)

    # On fusionne les deux moitiés triées
    return merge(left, right)

def merge(left, right):
    result = []
    i = j = 0

    # Tant que les deux listes contiennent des éléments
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1

    # Si une des listes contient encore des éléments, on les ajoute tous à la liste résultat
    while i < len(left):
        result.append(left[i])
        i += 1
    while j < len(right):
        result.append(right[j])
        j += 1

    return result


# Merge sort avec Threads

In [2]:
import threading

# Fonction pour trier avec des threads
def tri_fusion_threads(lst):
    if len(lst) <= 1:
        return lst

    mid = len(lst) // 2
    left_half = lst[:mid]
    right_half = lst[mid:]

    left_thread = threading.Thread(target=merge_sort, args=(left_half,))
    right_thread = threading.Thread(target=merge_sort, args=(right_half,))

    left_thread.start()
    right_thread.start()

    left_thread.join()
    right_thread.join()

    return merge(left_half, right_half)

# Merge sort avec processus

In [3]:
import multiprocessing

# Fonction pour trier avec des processus
def tri_fusion_processes(lst):
    if len(lst) <= 1:
        return lst

    mid = len(lst) // 2
    left_half = lst[:mid]
    right_half = lst[mid:]

    with multiprocessing.Pool(2) as pool:
        left_half = pool.apply(merge_sort, (left_half,))
        right_half = pool.apply(merge_sort, (right_half,))

    return merge(left_half, right_half)

In [13]:
import time

#D'efinition de la liste a trier
lst = [38, 27, 43, 3, 9, 82, 10,78,66,23,2,79]

# Tri séquentiel
start_time = time.time()
sorted_list_sequential = merge_sort(lst.copy())
end_time = time.time()
print("Tri avec séquentiel\n\t" , sorted_list_sequential)
print("\tTemps d'exécution (en secondes) :", end_time - start_time)
print("=" * 65)

# Tri avec threads
start_time = time.time()
sorted_list_threads = tri_fusion_threads(lst.copy())
end_time = time.time()
print("Tri avec threads\n\t" , sorted_list_threads)
print("\tTemps d'exécution (en secondes) :", end_time - start_time)
print("=" * 65)

# Tri avec processus
start_time = time.time()
sorted_list_processes = tri_fusion_processes(lst.copy())
end_time = time.time()
print("Tri avec processus\n\t" , sorted_list_processes)
print("\tTemps d'exécution (en secondes) :", end_time - start_time)


Tri avec séquentiel
	 [2, 3, 9, 10, 23, 27, 38, 43, 66, 78, 79, 82]
	Temps d'exécution (en secondes) : 9.560585021972656e-05
Tri avec threads
	 [10, 38, 27, 43, 3, 9, 78, 66, 23, 2, 79, 82]
	Temps d'exécution (en secondes) : 0.00661921501159668
Tri avec processus
	 [2, 3, 9, 10, 23, 27, 38, 43, 66, 78, 79, 82]
	Temps d'exécution (en secondes) : 0.030514001846313477


# Analyse des resultats
Les résultats dépendent de la longueur de la liste donnée et du matériel utilisé.
En principe: le tri séquentiel est le plus lent, suivi du tri avec des threads (en raison du GIL de Python), tandis que le tri avec des processus peut être le plus rapide, en particulier lorsque plusieurs cœurs de CPU sont disponibles.

**Impact du coût de la création de threads ou de processus :**

La création de threads est généralement moins coûteuse que la création de processus, car les threads partagent la mémoire de leur processus parent. Cependant, cela dépend du système d'exploitation et du langage de programmation utilisés.
La création de processus est plus coûteuse en termes de temps et de mémoire, car chaque processus a sa propre mémoire et son propre espace d'adressage.

**Impact du GIL (Global Interpreter Lock) :**

Le GIL de Python limite l'exécution concurrente de plusieurs threads dans le même processus Python. Cela signifie que, bien que les threads puissent améliorer les performances dans certains cas, ils ne peuvent pas exploiter pleinement plusieurs cœurs de processeur.
En conséquence, les threads Python ne sont pas toujours aussi efficaces pour des tâches fortement parallèles, ce qui limite leur avantage par rapport aux processus

# Partie comparaison de plusieurs threads et processus

# Merge sort qui supporte plusieurs threads

In [14]:
import threading

# Fonction pour trier avec des threads
def tri_avec_threads(lst, num_threads):
    part_size = len(lst) // num_threads
    threads = []
    partie_trie = []

    # Fonction pour trier un morceau de la liste en parallèle
    def trierPartie(part):
        sorted_part = merge_sort(part)
        partie_trie.append(sorted_part)

    # Création et démlstage des threads
    for i in range(num_threads):
        start = i * part_size
        end = start + part_size if i < num_threads - 1 else len(lst)
        part = lst[start:end]
        thread = threading.Thread(target=trierPartie, args=(part,))
        threads.append(thread)
        thread.start()

    # Attente de la fin de tous les threads
    for thread in threads:
        thread.join()

    # Fusion des morceaux triés
    sorted_lst = partie_trie[0]
    for i in range(1, num_threads):
        sorted_lst = merge(sorted_lst, partie_trie[i])

    return sorted_lst



# Merge sort qui supporte plusieurs processus

In [15]:
import multiprocessing

# Fonction pour trier avec des processus
def tri_avec_processus(lst, num_processes):
    part_size = len(lst) // num_processes
    processes = []
    partie_trie = []

    # Fonction pour trier un morceau de la liste en parallèle
    def trierPartie(part, output):
        sorted_part = merge_sort(part)
        output.put(sorted_part)

    # Création et démlstage des processus
    for i in range(num_processes):
        start = i * part_size
        end = start + part_size if i < num_processes - 1 else len(lst)
        part = lst[start:end]
        output = multiprocessing.Queue()
        process = multiprocessing.Process(target=trierPartie, args=(part, output))
        processes.append((process, output))
        process.start()

    # Attente de la fin de tous les processus
    for process, output in processes:
        process.join()
        partie_trie.append(output.get())

    # Fusion des morceaux triés
    sorted_lst = partie_trie[0]
    for i in range(1, num_processes):
        sorted_lst = merge(sorted_lst, partie_trie[i])


    return sorted_lst

Partie main du programme pour l' affichage du temps de chaque fonction de multithreads et multiprocessus

In [16]:
import time

# Nombre de threads
num_threads = 2
# Nombre de processus
num_processes = 2

lst = lst = [38, 27, 43, 3, 9, 82, 10,78,66,23,2,79]


# Tri avec des threads
start_time = time.time()
sorted_list_threads = tri_avec_threads(lst.copy(), num_threads)
end_time = time.time()
print("Tri avec threads\n\t" , sorted_list_threads)
print("\tTemps d'exécution (en secondes) :", end_time - start_time)
print("=" * 65)

# Tri avec des processus
start_time = time.time()
sorted_list_processes = tri_avec_processus(lst.copy(), num_processes)
end_time = time.time()
print("Tri avec processus\n\t" , sorted_list_processes)
print("\tTemps d'exécution (en secondes) :", end_time - start_time)

Tri avec threads
	 [2, 3, 9, 10, 23, 27, 38, 43, 66, 78, 79, 82]
	Temps d'exécution (en secondes) : 0.0005075931549072266
Tri avec processus
	 [2, 3, 9, 10, 23, 27, 38, 43, 66, 78, 79, 82]
	Temps d'exécution (en secondes) : 0.03475809097290039


l'impact du tri fusion parallèle sur les performances dépend de plusieurs facteurs, notamment la taille des données, le nombre de cœurs ou de processeurs disponibles, la complexité de la fusion, la gestion de la mémoire et la synchronisation. Pour de grandes quantités de données avec un matériel adapté, le tri fusion parallèle peut offrir une amélioration significative des performances par rapport à l'exécution séquentielle. Cependant, pour de petites quantités de données ou avec des contraintes de ressources, l'utilisation de plusieurs threads ou processus peut ne pas être aussi bénéfique et peut même entraîner une perte de performance due à l'overhead de la gestion concurrente.